# Volumeter

TODO
- measurementCycle(loweringSpeed)
- calibrateHolder()
- syncMeasurementData() # Needs to incorporate some sort of interpolation to sync based on immersion depth insted of samples. Maybe the volume should be calculated in this function.
- ~calculateVolume()
- visualize()
- exportCSV()

In [4]:
import serial
import pandas as pd
import numpy as np
import time
import os
from IPython.core.display import clear_output

In [ ]:
global loadCellConstants
global mcSerial
global sensorSerial
global homingDone

def readLoadCellCalibration():
    '''Loads the load cell calibration values from a CSV file called loadCellCalibration.csv.'''
    cal = np.genfromtxt('loadCellCalibration.csv', delimiter=',')
    return cal

def loadCellToGrams(value):
    '''Converts a load cell reading to grams based on the calibration values.'''
    return value*loadCellConstants[0] + loadCellConstants[1]

def initMotionControl(motionControlPort='/dev/ttyUSB1'):
    '''Initializes the motion control serial port.
    
    Uses a default of /dev/ttyUSB1 but the port can be given as an argument.
    '''
    global mcSerial
    global homingDone
    mcSerial = serial.Serial(port=motionControlPort, baudrate=115200)
    homingDone = False

def initSensors(sensorPort='/dev/ttyUSB0'):
    '''Initializes the sensor serial port.
    
    Uses a default of /dev/ttyUSB0 but the port can be given as an argument.
    '''
    global sensorSerial
    global loadCellConstants
    loadCellConstants = readLoadCellCalibration()
    
    sensorSerial = serial.Serial(port=sensorPort, baudrate=115200)

def closeSerial(serial):
    '''Closes the serial port that was given as an argument'''
    serial.close()

def sendMotionCommand(cmd):
    '''Sends the given gcode command to the motion control serial port.'''
    mcSerial.write('{0}\n'.format(cmd).encode('utf-8'))
    mcSerial.flushInput()
    grbl_out = mcSerial.readline()
    return grbl_out.decode()

def motionFinished(wait=False):
    '''Returns True if the motion control system is idle
    
    Giving wait=True as an argument waits until the motion control system becomes idle and returns True.
    '''
    finished = False
    if(wait):
        while(not finished):
            time.sleep(0.5)
            grbl_out = sendMotionCommand('?')
            finished = 'Idle' in str(grbl_out)
        return True
    else:
        grbl_out = sendMotionCommand(serial, '?')
        return 'Idle' in str(grbl_out)

def homingCycle():
    '''Homes the motion control system axes.'''
    
    global homingDone
    
    cycle = ['?', 'G91', 'G38.2Z2500F2000', 'G1Z-20F1000',
             'G38.2X-560Z-560F300', 'G1X10Z10F300', 'G10L20P1X0Y0Z0',
            'G90', 'G1Z-1650F3000']
    for c in cycle:
        sendMotionCommand(c)
    motionFinished(wait=True)
    homingDone = True

def recordSensors(numSamples):
    '''Records the sensors for the given number of samples and return the in a pandas DataFrame.'''
    sensorSerial.flushInput()
    samplesRead = 0
    df = pd.DataFrame(columns=['time', 'loadCell', 'level'], dtype='float')
    while(samplesRead < numSamples):
        try:
            data = np.array(sensorSerial.readline().decode().rstrip().split(','))
            data = data.astype(np.float)
            df.loc[len(df)] = data
            samplesRead += 1
        except:
            pass
    df.set_index(['time'])
    return df

def meanLoadCell(numSamples):
    '''Returns the mean load cell value of given number of samples'''
    data = recordSensors(numSamples)
    return data['loadCell'].mean()

def meanWeight(numSamples):
    '''Returns the mean weight in grams of given number of samples'''
    data = recordSensors(numSamples)
    mean = data['loadCell'].mean()
    return loadCellToGrams(mean)

def calibrateLoadCell(numWeights=1):
    '''Runs a load cell calibration wizard.
    
    numWeights argument can be used to tell how many calibration weights you want to use.
    The calibration values are saved to loadCellCalibration.csv if the user accepts the new calibration values.
    '''
    grams = []
    values = []
    print("Load Cell Calibration")
    print("Leave the specimen holder empty")
    input("Hit enter when the specimen holder is stationary")
    print("Wait for 5 seconds...")
    grams.append(0.0)
    values.append(meanLoadCell(80*5))
    while(numWeights > 0):
        numWeights -= 1
        while(True):
            try:
                calWeight = input("Mass of the calibration weight in kg: ").rstrip().replace(',', '.')
                grams.append(float(calWeight)*1000.0)
                break
            except:
                print("Insuitable mass value")

        print("Add the calibratrion weight to the specimen holder")
        input("Hit enter when the specimen holder is stationary")
        print("Wait for 5 seconds...")
        values.append(meanLoadCellmeanWeight(80*5))
    calibrationConstants = np.polyfit(values, grams, 1)
    print("Calibration: y = {:.2f}x+{:.2f}".format(calibrationConstants[0], calibrationConstants[1]))
    save = input("Save calibration (y/n)")
    if('y' in save.lower()):
        np.savetxt('loadCellCalibration.csv', calibrationConstants, delimiter=',')
        print('Calibration saved')
    else:
        print('Calibration cancelled')
        
def calibrateHolder():
    '''Runs a specimen holder calibration cycle and asks the user for a name of the calibration.

    Calibration values are saved to holderCalibrations.csv file whose column specifies the name of the holder.
    '''
    if(not homingDone):
        print("Motion control system is not homed. Please run the homingCycle() first.")
        return None

    motionFinished(wait=True)

    movementsToTopOfTank = ['?', 'G90', 'G1X0Z-50F4000', 'G1X540Z540F3000','G1X540Z300F2000']
    for c in movementsToTopOfTank:
        sendMotionCommand(c)
    motionFinished(wait=True)

def selectSpecimenHolder():
    '''Lists the names of the available specimen holder calibrations and asks the user to select one of them.'''

def measurementCycle():
    '''Runs the measurement cycle.'''
    if(not homingDone):
        print("Motion control system is not homed. Please run the homingCycle() first.")
        return None

    motionFinished(wait=True)

    
        

In [ ]:
initMotionControl()
finished = motionFinished(wait=True)
homingCycle()
#closeSerial(mcSerial)

In [6]:
initSensors()
#data = recordSensors(10)
for i in range(10):
    mw = meanWeight(20)
    clear_output(wait=True)
    print('{:.2f}'.format(mw))
#mw = meanWeight(10)
#calibrateLoadCell(numWeights = 2)
closeSerial(sensorSerial)
#print(mw)

178.97


In [12]:
calibrateHolder()
closeSerial(mcSerial)

In [ ]:
os.system('sudo shutdown.sh')